# Build Functions Script
Name is WIP

## Goals:
- Build a deployable script that can reference and update an online list of functions, as well as build a .py file with these functions

## Stakeholders:
- Mostly me, but it could prove to be super useful to others too

## Deliverables:
- function viewer + editor:
    - create a dictionary object containing details about a function
    - create a list of dictionary objects, where each item is a function
    - edit and remove functions
    - show information about a function (to include dependencies and syntax)
- library viewer + editor:
    - load a list of Python libraries and modules
    - show information about these libraries and modules
    - add and edit these libraries and modules
- module builder:
    - get from the user the name of the file to create
    - check if the file given matches any tags associated with the functions list
    - suggest functions based on their tags
- function assembler:
    - get functions to include from the user
    - assemble a list of functions 
- library assembler:
    - get dependencies for a given function
    - assemble a list of dependencies for a given function (probably using a tuple)
- grand function:
    - runs the above in some fashion

In [2]:
# # functions example:
# functions = [
# 	{
# 		'function_name':'function_call_name',
# 		'modules':['module for function']
# 		'description':'description of function actions',
# 		'dependencies':{
# 			'name':'type'
# 		},
# 		'syntax':'code to create the function''
# 	}
# ]

# # libraries example
# libraries = [
# 	{
# 		'name':'library/module name',
# 		'syntax':'library/module syntax'
# 	}
# ]

In [3]:
# establish imports
import re
import pandas as pd

# import function files
import base
from library import manage_library
from function import manage_functions
from assembly import assembler

In [32]:
manage_library()

What would you like to do?



        1) Add a new entry
        2) Edit an existing one
        3) View a list of current libraries
        4) View a specific library
        5) Exit
         5


In [142]:
manage_functions()

What would you like to do?



        1) Add a new entry
        2) Edit an existing one
        3) Remove a specific function
        4) View a list of current functions
        5) View a specific function
        6) Exit
         6


In [93]:
assembler()

Current list of entries:
df_info: Function takes a dataframe and returns potentially relevant information about it
print_libs: Function that prints all libraries used up to present
check_file_exists: Function takes a filename, query, and url and checks if the file exists
drop_extras: Function to drop extra columns that may have a smaller impact on the model
split_categorical: Returns three dataframes split from one for use in model training, validation, and testing
drop_cols: Drops columns
encode_df: Takes a processed dataframe and encodes the object columns for usage in modeling



Enter function names separated by commas:  df_info, check_file_exists, drop_extras, split_categorical


Selected functions: df_info, check_file_exists, drop_extras, split_categorical


Press Enter to confirm or 'restart' to start over:  


Creating wrangle.py...


### Function 0.1: Data Saver
**Idea**: This function will be the crux of operations. Most operations would involve this function, as they require the data stored within.

**Final Product**: A function that by default returns None. If no arguments are given, it does nothing. If a DataFrame is provided, then one of its other arguments, 'library' or 'function' must be True. If 'library' or 'function' is True, then it will perform the operations of loading the specific data and returning it.

Parameters:
- load_library=False
    - if True then checks if library csv exists. If library csv exists, then load and return it. If it does not exist, then make it and return starter DataFrame with 'name', 'syntax', and 'desc' columns.
- load_function=False
    - if True, then checks if functions json exists. If functions json exists, then load and return it. If it does not exist, then make it and return starter DataFrame with the 'name','desc','syntax', and 'dependencies' columns.
- saved_data=None
    - if not None, then checks number of columns. If length of columns matches library list, then overwrite libraries.csv. If length of columns matches functions list, then overwrite functions.json. Else print that length matches neither.

In [9]:
def data_saver(load_library=False,load_function=False,saved_data=None):
    # check if load_library is true
    if load_library == True:
        # check if libraries.csv exists
        if os.path.exists('libraries.csv') == True:
            libraries = pd.read_csv('libraries.csv',index_col=0)
            return libraries
        else:
            libraries = pd.DataFrame(columns=['name','syntax','desc'])
            libraries.to_csv('libraries.csv')
            return libraries
    
    # check if load_function is true
    elif load_function == True:
        if os.path.exists('functions.json') == True:
            functions = pd.read_json('functions.json')
            return functions
        else:
            functions = pd.DataFrame(columns=['name','desc','tags','dependencies','syntax'])
            functions.to_json('functions.json')
            return functions
    
    # check if saved_data is not None
    elif type(saved_data) == type(pd.DataFrame()):
        if len(saved_data.columns) == 3:
            saved_data.to_csv('libraries.csv')
        elif len(saved_data.columns) == 5:
            saved_data.to_json('functions.json')
        else:
            print('Saved data length matches none')
    
    else:
        print('No options provided. Exiting.')
    
    return None

In [11]:
data_saver(load_library=True)

,name,syntax,desc
0,pandas,import pandas as pd,data manipulation library


### Function 0.2: Format text with embedded newlines
**Problem:** So at the function editor, I encountered the problem that copying and pasting a function results in a mess of formatting. Newlines aren't kept, so everything is globbed together in an ugly mess. This won't work for when the function begins to build the modules, so I need to correct this issue now so that everything will format correctly.

**Solution:** The following function relies on the *re* module. It finds any 3 or more consecutive spaces and adds a newline character to the string before the spaces. This way, any text entered will, when printed, be written with the appropriate indentations and should work as intended.

In [48]:
def add_newline_for_spaces(text):
    # Replace two or more spaces with a newline followed by the spaces
    modified_text = re.sub(r'(\S)(\s{3,})', r'\1\n\2', text)
    return modified_text

### Function 0.3: Check current OS

In [162]:
import platform

def check_operating_system():
    os_name = platform.system()
    if os_name == "Windows":
        return "Windows"
    elif os_name == "Darwin":
        return "macOS"
    elif os_name == "Linux":
        return "Linux"
    else:
        return "Unknown Operating System"

In [164]:
running_os = check_operating_system()
running_os

'macOS'

### Function 0.4: Add builder command to OS

In [167]:
import os
from IPython.core.interactiveshell import InteractiveShell

def load_aliases(current_dir):
    alias_cmd = f"python {current_dir}/build_a_mod.py"
    shell = InteractiveShell.instance()
    shell.define_alias('funct_build', alias_cmd)


In [169]:
import sys

def add_command_to_shell():
    # Determine the shell configuration file based on the user's current shell
    shell_config = {
        '/bin/bash': '~/.bashrc',
        '/bin/zsh': '~/.zshrc'
    }.get(os.environ.get('SHELL'), None)

    if not shell_config:
        print("Unsupported shell or shell not detected.")
        return
    
    # Expand the user path
    shell_config_path = os.path.expanduser(shell_config)

    # Command to be added
    current_dir = os.getcwd()  # Get the current working directory
    command_str = f"alias funct_build='python {current_dir}/build_a_mod.py'\n"

    # Check if the command already exists in the file
    try:
        with open(shell_config_path, 'r') as file:
            if command_str in file.read():
                print("Command already exists in the shell configuration.")
                return
    except FileNotFoundError:
        print(f"No such file: {shell_config_path}")
        return

    # Append the command to the shell configuration file
    try:
        with open(shell_config_path, 'a') as file:
            file.write(command_str)
            print(f"Command added to {shell_config}. You may need to restart your shell or source your {shell_config} file.")
    except Exception as e:
        print(f"Failed to add command to {shell_config}: {str(e)}")
    
    try:
        load_aliases(current_dir)
    except:
        print('Failure')

In [171]:
if (running_os=='macOS')|(running_os=='Linux'):
    add_command_to_shell()
elif running_os=='Windows':
    print("I'm sorry -- this Operating System is currently not supported for adding commands.")
else:
    print("I'm sorry -- this Operating System is not recognized.")

Command already exists in the shell configuration.


In [28]:
os.getcwd()

'/Users/manelson/github_projects/Build-Functions'

### Final product

In [147]:
def sleep_end(t_bed):
    if t_bed+7 <=7:
        return t_bed+7
    else:
        sleep_n = 7-t_bed
        sleep_m = (7-sleep_n)/2
        return t_bed + sleep_n + sleep_m

In [153]:
sleep_end(5)

9.5